In [ ]:
import os, csv, re, glob, math, collections, sys, codecs
import xml.etree.ElementTree as ET
from matplotlib import pylab as plt
import matplotlib.pyplot as plot
import pandas as pd
from io import BytesIO
import numpy as np
import pandas as pd
from scipy import signal
from scipy.signal import find_peaks
from scipy import stats
from colorama import *

In [ ]:
print(Fore.BLUE + '####################################\n# Welcome to the SalivaPrint tool. #\n#----------------------------------#\n#        Type [y] to start.        #\n####################################')
choice=''
choice = input()
if choice=='y':
    print(Fore.RED + '-> Data Extraction from XML file.<-\n')
else:
    sys.exit(Fore.RED + 'Stop')
#-Next step is add a condition to input the Sample ID and confirm if is the of XML file to continue-#

In [ ]:
# Script1
# Createa csv data from xml source
full_xml_path = os.path.abspath(os.path.join(".", "test.xml"))
full_csv_path = os.path.abspath(os.path.join(".", "Samples.csv"))


tree = ET.parse(full_xml_path)
root = tree.getroot()

file_header = ['Sample']
file_header_vals = []
sample_well_info = []


In [ ]:
# Set File Header
for i in range(466, 869):
    file_header.append(str(round((0.00042006 * (i * i) - 0.28216 * i + 49.225), 1)))
    file_header_vals.append(str(round((0.00042006 * (i * i) - 0.28216 * i + 49.225), 1)))

In [ ]:
# get all well id's and sample names from file
for sample in root.findall('Results'):
    # write sample name
    well_id = sample.find('WellId').text

    if well_id != '0':
        sample_name = sample.find('SampleName').text
        sample_well_info.append({'Sample': sample_name, 'well_id': well_id})

In [ ]:
# create file with headers and data
with open(full_csv_path, 'w', newline='') as outcsv:
    writer = csv.DictWriter(outcsv, fieldnames = file_header)
    writer.writeheader()
    xmlstr = ET.tostring(root, encoding='utf8', method='xml').decode("utf-8")

    for elem in sample_well_info:
        
        idx = 0
        row = {'Sample': elem['Sample']}
        regex = '<RawDataPoints><WellId>%s<\/WellId><Signal>(.*?)<\/Signal><\/RawDataPoints>'% elem['well_id']
        raw_data_points = re.findall(regex, xmlstr)
        
        # write data for each column
        for count, raw_data_point in enumerate(raw_data_points, start=0):
            if 869 >= count >= 467:
                row[file_header_vals[idx]] = raw_data_point
                idx+=1
            
            if count > 869:
                break
        
        writer.writerow(row)

In [ ]:
df = pd.read_csv('Samples.csv', index_col='Sample')
df.head(2)

df2 = df.T
df2.to_csv('output.csv')
df2.head(2)

ax = df2.plot()
ax.set_xlabel('Molecular Weight (kDa)')
ax.set_ylabel('Fluorescence')

In [ ]:
#Do you want to plot
print(Fore.GREEN + '#########################################\n#    Do you want to plot the profile?   #\n#---------------------------------------#\n#  Type [y] to plot or [n] to continue. #\n#########################################')
choice = ''
choice = input()
if choice == 'y':
    print(Fore.RED + 'Line Graph Plot.')
    plt.show()
elif choice == 'n':
    print(Fore.BLUE + 'Continue the profile analysis.')
else:
    sys.exit('Stop')
